In [1]:
import sys
sys.path.append(".")
sys.path.append("..")

import typing as t
import ratline as rl
from pydantic import field_validator

In [2]:
from tools import DotNetReversingTool
from pathlib import Path

binaries_to_reverse: dict[str, Path] = {
    "AddInUtil.exe": Path("binaries/AddInUtil.exe"),
    "System.Addin.dll": Path("binaries/System.Addin.dll"),
}

reversing_tool = DotNetReversingTool(binaries_to_reverse)

In [3]:
# Common models

class Thinking(rl.CoreModel, tag="thinking"):
    content: str

class Answer(rl.CoreModel, tag="answer"):
    content: str

class Description(rl.CoreModel, tag="description"):
    content: str

class YesNoAnswer(rl.CoreModel, tag="yes_no_answer"):
    value: bool

    @field_validator('value', mode='before')
    def parse_str_to_bool(cls, v):
        if isinstance(v, str):
            if v.strip().lower() == 'yes':
                return True
            elif v.strip().lower() == 'no':
                return False
        return v

In [4]:
# Common prompts

ResearcherSystemMessage: rl.MessageDict = {
    "role": "system",
    "content": "You are a talented security researcher experienced in complex vulnerability research for Windows.",
}


def method_vulnerability_message(code: str) -> rl.MessageDict:
    prompt = f"""\
Here is the code you are analyzing:

<code>{code}</code>

You will analyze this code for vulnerabilities related to .NET deserialization. This method has been identified by searching for references to specific function keywords in a target .NET assembly. Answer whether you believe this code this vulnerable to deserialization attacks and place your yes/no answer between {YesNoAnswer.xml_tags()} tags. If you believe this code is vulnerable, please describe the vulnerability between {Description.xml_tags()} tags. Do not provide any mitigation advice at this time.

Before answering,document your thoughts between {Thinking.xml_tags()} tags.
"""
    return {
        "role": "user",
        "content": prompt,
    }

In [5]:
# Exporation - 1

explore_1 = rl.make_study("explore_1", "anthropic:model=claude-coast-1.0", max_iters=5)


@explore_1.exp()
def get_interesting_functions(generator: rl.Generator):
    rl.tag("prompt_version", 1)

    messages: list[rl.MessageDict] = [
        ResearcherSystemMessage,
        {
            "role": "user",
            "content": f"Provide a list of function search strings that could be used during reversing to identify instances of deserialization vulnerabilities in .NET assemblies. These function search strings will be passed to a tool which will list all functions in a .NET assembly that use dependencies matching the search string. Before you begin, document your thoughts in {Thinking.xml_tags()} tags. When you are ready, list your function search strings in as comma-delimited list inside {Answer.xml_tags()} tags.",
        },
    ]
    chat = generator.chat(messages).until_parsed_as(Thinking, Answer).run()

    answer = chat.last.parse(Answer)
    thinking = chat.last.parse(Thinking)

    functions = [s.strip() for s in answer.content.split(",")]
    rl.log(f"Searching for functions: {','.join(functions)}")

    references: dict[str, list[str]] = {}
    for function in functions:
        for binary_name in binaries_to_reverse.keys():
            if binary_name not in references:
                references[binary_name] = []
            references[binary_name].extend(reversing_tool.find_references_to(binary_name, function).split('\n'))

    for name, refs in references.items():
        references[name] = list(set(refs))
        rl.log(f"Found {len(references[name])} references in {name}")

    total_references = sum([len(refs) for refs in references.values()])
    rl.tag("reference_count", total_references)

    rl.keep("functions", functions)
    rl.keep("references", references)
    rl.keep("thoughts", thinking.content)

In [ ]:
explore_1.run()

In [6]:
# Exploration - 2

explore_2 = rl.make_study("explore_2", "anthropic:model=claude-coast-1.0", max_iters=1)

@explore_2.inspect("explore_1.get_interesting_functions")
def inspect(trials: Trial):
    if 
    pass

@explore_2.foreach("explore_1.get_interesting_functions")
def thing():
    pass

@explore_2.exp_using(
    "explore_1.get_interesting_functions",
    when_tags=lambda tag: tag.content == "reference_count" and (tag.value or 0) > 0,

)
def inspect_interesting_functions(interesting_function, generator: rl.Generator):
    references: dict[str, list[str]] = rl.recall("references")

    for binary_name, functions in references.items():
        for function in functions:
            if not function:
                continue

            function_code = reversing_tool.decompile_method(binary_name, function)
            chat = (
                generator.chat(
                    [
                        ResearcherSystemMessage,
                        method_vulnerability_message(function_code),
                    ]
                )
                .until_parsed_as(Thinking, YesNoAnswer)
                .run()
            )

            answer = chat.last.parse(YesNoAnswer)
            thinking = chat.last.parse(Thinking)

            rl.keep("thoughts", thinking.content)

            if not answer.value:
                rl.log(f"Function {function} is not vulnerable")
                continue

            rl.log(f"Function {function} is vulnerable")
            description = chat.last.parse(Description)
            rl.log(f"Vulnerability description: {description.content}")

In [7]:
explore_2.run()

[=] 
[=] Starting study 'explore_2'
[=] 
[=] Checking dependencies for experiments (1)
[=]  |- 'inspect_interesting_functions/explore_1.get_interesting_functions:latest' -> 5/10 trials
[=] 
[+] 
[+] New run 'shiny-cat' started for study 'explore_2'
[+] 
[=] explore_2.inspect_interesting_functions (iter 1/1)
[=]  |  > Trial 89 [using: 64] (1/5)
[=]  |     |: Function T System.AddIn.Hosting.AddInActivator::ActivateInAppDomain(System.AddIn.Hosting.AddInToken,System.AppDomain,System.AddIn.Hosting.AddInControllerImpl,System.Boolean) is not vulnerable
[=]  |     |: Function System.AddIn.Hosting.AddInServerWorker System.AddIn.Hosting.AddInServer::CreateDomain(System.AddIn.Hosting.AddInToken,System.Security.PermissionSet) is vulnerable
[=]  |     |: Vulnerability description: 
This code is vulnerable to .NET deserialization attacks in a few ways:

1. AppDomainSetup.ConfigurationFile allows specifying an arbitrary config file that the new AppDomain will load. This file could contain malicious c

'shiny-cat'

In [ ]:


@rl.experiment
def do_thing(ctx: rl.Context):
    generator = ctx.get(rl.Generator)
    
    system_message = trial.suggest("system_researcher_prompt")
    trial.keep("thing", 123)


@do_thing.inspect_all()
def inspect(trials: list[rl.Trial]):
    pass

@do_thing.inspect()
def inspect_single(trial: rl.Trial):
    pass

@do_thing.for_all()
def do_other_thing(trial: rl.Trial):
    trial.recall("thing")

rl.study(do_thing) \
    .when(lambda trial: trial.thing == 123, do_other_thing) \
    .when(lambda trial: trial.)
    .then(do_other_thing).where(lambda trial: trial.).run()
